# Config

In [1]:
import pandas as pd
import json
import os

In [10]:
# Example usage
path = "../data/pilot_static_data.csv"  # Replace with the actual path
kpi_path = "../data/cities_kpi.csv"  # Replace with the actual path

output_general = "../config/pilots.json"
output_cities = "../data/pilots/"
output_kpis = "../config/kpi.json"

# Read data

In [ ]:
df = pd.read_csv(path,sep="\t")
df.head(1)

,City,Pilot,Thematic Clasters,Impact Areas,Start Date Q,End Date Q,Start Date,End Date,Duration (Months),Description,...,KPIs,Link to Dashboard,UMI Categories,Sources,Tips for replication,Pictures,lat,lon,name,location
0,Gothenburg,Safe System Approach for Children’s Active Tra...,Safe & Sustainable Schools,"Road Safety, Transformative Governance, Inclus...",Q2 2023,Q4 2025,04/01/2023,31/12/2025,35,In the neighbourhood of Bergum Gunnilse key mo...,...,"1, 2, 9",NaN,Modal Share,NaN,NaN,https://reallocatemobility.eu/fileadmin/user_u...,"57,70887","11,97456",Gothenburg (pilot 1),Bergum Gunnilse


In [9]:
kpi = pd.read_csv(kpi_path,sep=";")
print(kpi.columns)
kpi = kpi[['KPI', 'Name', 'Category', 'Indicator', 'Unit']]
print(kpi.columns)

kpi.head()

Index(['KPI', 'Name', 'Unnamed: 2', 'Category', 'Indicator', 'Unit'], dtype='object')
Index(['KPI', 'Name', 'Category', 'Indicator', 'Unit'], dtype='object')


,KPI,Name,Category,Indicator,Unit
0,1,Modal Share,Modal Share,Measure the percentage of commuters using acti...,Percentage (%)
1,2,Number of Participants in Co-Creation Workshops,Engagement,Count the total number of individuals who acti...,Count
2,3,Conflicts Removed or Mitigated,Accidents & Injuries,Number of conflicts that have been removed or ...,Count
3,4,Service coverage of Demand-Responsive Transpor...,Access to mobility services,NaN,NaN
4,5,Replicability of Bicycle Policies,Access to mobility services,The percentage of other road sections or regio...,Percentage (%)


# General JSON

In [27]:
def extract_all_pilots(df, output_json_path):
    pilots = []
    for _, row in df.iterrows():
        pilot_id = row['name'].lower().replace(" ", "_").replace("(", "").replace(")", "")
        start_date = row["Start Date"]
        if isinstance(start_date, pd.Timestamp):
            start_date = start_date.strftime("%Y-%m-%d")

        # Handle end_date override (if not in standard format)
        end_date = row["End Date"]
        if isinstance(end_date, pd.Timestamp):
            end_date = end_date.strftime("%Y-%m-%d")

        pilot_data = {
            "id": pilot_id,
            "city": row["City"],
            "title": row["Pilot"],
            "name": row["name"],
            "start_date": start_date,
            "end_date": end_date,
            "image": row["Pictures"],
            "lat": row["lat"],
            "lon": row["lon"],
            "thematic": row["Thematic Clasters"],
            "impact_areas": row["Impact Areas"],
            "kpis": row["KPIs"],
            "description": row["Description"]
        }

        pilots.append(pilot_data)

    # Save to JSON
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(pilots, f, ensure_ascii=False, indent=2)

    print(f"Saved {len(pilots)} pilots to {output_json_path}")


In [28]:
extract_all_pilots(df, output_general)

Saved 15 pilots to ../config/pilots.json


# City Json

In [29]:
def save_pilots_as_individual_jsons(df, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for _, row in df.iterrows():
        # Generate pilot ID from name
        pilot_id = row["name"].lower().replace(" ", "_").replace("(", "").replace(")", "")

        # Parse KPIs into a list
        kpis = [k.strip() for k in str(row["KPIs"]).split(",") if k.strip().isdigit()]

        # Construct JSON structure
        pilot_json = {
            "id": pilot_id,
            "city": row["City"],
            "title": row["Pilot"],
            "image": row["Pictures"],
            "tabs": {
                "Pilot Info": {
                    "description": row["Description"],
                    "location": row["location"],
                    "lat": row["lat"],
                    "lon": row["lon"],
                },
                "KPIs": kpis,
                "Dashboard": {
                    "title": "",
                    "description": "",
                    "url": row.get("Link to Dashboard", None)
                }
            }
        }

        # Save to file
        filename = f"{pilot_id}.json"
        filepath = os.path.join(output_folder, filename)
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(pilot_json, f, ensure_ascii=False, indent=2)

        print(f"Saved: {filepath}")

In [30]:
save_pilots_as_individual_jsons(df, output_cities)

Saved: ../data/pilots/gothenburg_pilot_1.json
Saved: ../data/pilots/gothenburg_pilot_2.json
Saved: ../data/pilots/heidelberg_pilot_1.json
Saved: ../data/pilots/heidelberg_pilot_2.json
Saved: ../data/pilots/barcelona_pilot_1.json
Saved: ../data/pilots/barcelona_pilot_2.json
Saved: ../data/pilots/budapest_pilot_1.json
Saved: ../data/pilots/budapest_pilot_2.json
Saved: ../data/pilots/lyon_pilot_1.json
Saved: ../data/pilots/lyon_pilot_2.json
Saved: ../data/pilots/bologna.json
Saved: ../data/pilots/utrecht.json
Saved: ../data/pilots/tampere.json
Saved: ../data/pilots/warsaw.json
Saved: ../data/pilots/zagreb.json


## KPI Jsons

In [11]:
def extract_all_kpis(df, output_json_path):
    kpis = []
    
    for _, row in df.iterrows():
        kpi_data = {
            "id": int(row["KPI"]),
            "name": row["Name"],
            "category": row["Category"],
            "indicator": row["Indicator"] if pd.notna(row["Indicator"]) else "",
            "unit": row["Unit"] if pd.notna(row["Unit"]) else ""
        }
        kpis.append(kpi_data)

    # Save to JSON
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(kpis, f, ensure_ascii=False, indent=2)

    print(f"Saved {len(kpis)} KPIs to {output_json_path}")


In [12]:
extract_all_kpis(kpi,output_kpis)

Saved 16 KPIs to ../config/kpi.json
